# CLIP with Flowers!?!?!??!?

In [ ]:
import os
# Prefer expandable segments to reduce fragmentation (restart kernel after changing)
os.environ.setdefault('PYTORCH_CUDA_ALLOC_CONF', 'expandable_segments:True')

import sys
import torch
import torchvision

# Ensure CLIP is installed in the current kernel; install if missing.
# Using subprocess with sys.executable to target the same Python interpreter.
try:
    import clip
except Exception:
    import subprocess, importlib
    try:
        get_ipython().run_line_magic('pip', 'install --upgrade git+https://github.com/openai/CLIP.git')
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "git+https://github.com/openai/CLIP.git"], stdout=subprocess.DEVNULL)
    importlib.invalidate_caches()
    import clip

from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import autocast, GradScaler


In [2]:
# Ambiente: verifica e installazione sicura di CLIP
import sys, importlib, torch

try:
    import clip
    print("CLIP: già installato")
except Exception:
    print("CLIP non trovato: eseguo installazione nel kernel corrente...")
    import importlib
    # Preferisci %pip per installare nel kernel Jupyter corrente; fallback a subprocess
    try:
        get_ipython().run_line_magic('pip', 'install --upgrade git+https://github.com/openai/CLIP.git')
    except Exception:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "git+https://github.com/openai/CLIP.git"])    
    importlib.invalidate_caches()
    import clip
    print("CLIP installato correttamente nel kernel corrente")

print("python:", sys.executable)
print("torch:", torch.__version__, "cuda_available:", torch.cuda.is_available())
# su mac con Apple Silicon, controlla MPS
try:
    print("mps_available:", torch.backends.mps.is_available())
except Exception:
    pass

print('Riavvia il kernel se necessario, poi esegui questa cella e procedi con l\'allenamento CoCoOp.')


CLIP: già installato
python: /usr/bin/python3
torch: 2.9.0+cpu cuda_available: False
mps_available: False
Riavvia il kernel se necessario, poi esegui questa cella e procedi con l'allenamento CoCoOp.


## Dataset Functions

We define utility functions for:
- **`get_data()`**: Load Flowers102 from torchvision
- **`base_novel_categories()`**: Split 102 classes into base (0-50) and novel (51-101)
- **`split_data()`**: Filter images for base/novel in each split

This simulates the real scenario: we have 51 seen classes during training (base) and 51 new ones (novel).


In [3]:
def get_data(data_dir="./data", transform=None):
    """Load Flowers102 train, validation and test sets."""
    train = torchvision.datasets.Flowers102(root=data_dir, split="train", download=True, transform=transform)
    val = torchvision.datasets.Flowers102(root=data_dir, split="val", download=True, transform=transform)
    test = torchvision.datasets.Flowers102(root=data_dir, split="test", download=True, transform=transform)
    return train, val, test

def base_novel_categories(dataset):
    all_classes = set(dataset._labels)
    num_classes = len(all_classes)
    base_classes = list(range(num_classes))[:num_classes//2]
    novel_classes = list(range(num_classes))[num_classes//2:]
    return base_classes, novel_classes

def split_data(dataset, base_classes):
    base_categories_samples = []
    novel_categories_samples = []
    base_set = set(base_classes)
    
    for sample_id, label in enumerate(dataset._labels):
        if label in base_set:
            base_categories_samples.append(sample_id)
        else:
            novel_categories_samples.append(sample_id)
    
    base_dataset = torch.utils.data.Subset(dataset, base_categories_samples)
    novel_dataset = torch.utils.data.Subset(dataset, novel_categories_samples)
    return base_dataset, novel_dataset

## Class Names and Dataset Loading

We load the names of 102 flower classes from Flowers102.

This is **critical** for CLIP:
- Creates prompts like "a photo of a **rose**, a type of flower"
- Each prompt is encoded by CLIP's text encoder
- Image features are compared against these text templates


In [4]:
_, _, tmp_test = get_data()
base_classes, novel_classes = base_novel_categories(tmp_test)

CLASS_NAMES = ["pink primrose", "hard-leaved pocket orchid", "canterbury bells", "sweet pea", "english marigold", "tiger lily", "moon orchid", "bird of paradise", "monkshood", "globe thistle", "snapdragon", "colt's foot", "king protea", "spear thistle", "yellow iris", "globe-flower", "purple coneflower", "peruvian lily", "balloon flower", "giant white arum lily", "fire lily", "pincushion flower", "fritillary", "red ginger", "grape hyacinth", "corn poppy", "prince of wales feathers", "stemless gentian", "artichoke", "sweet william", "carnation", "garden phlox", "love in the mist", "mexican aster", "alpine sea holly", "ruby-lipped cattleya", "cape flower", "great masterwort", "siam tulip", "lenten rose", "barbeton daisy", "daffodil", "sword lily", "poinsettia", "bolero deep blue", "wallflower", "marigold", "buttercup", "oxeye daisy", "common dandelion", "petunia", "wild pansy", "primula", "sunflower", "pelargonium", "bishop of llandaff", "gaura", "geranium", "orange dahlia", "pink-yellow dahlia?", "cautleya spicata", "japanese anemone", "black-eyed susan", "silverbush", "californian poppy", "osteospermum", "spring crocus", "bearded iris", "windflower", "tree poppy", "gazania", "azalea", "water lily", "rose", "thorn apple", "morning glory", "passion flower", "lotus", "toad lily", "anthurium", "frangipani", "clematis", "hibiscus", "columbine", "desert-rose", "tree mallow", "magnolia", "cyclamen", "watercress", "canna lily", "hippeastrum", "bee balm", "ball moss", "foxglove", "bougainvillea", "camellia", "mallow", "mexican petunia", "bromelia", "blanket flower", "trumpet creeper", "blackberry lily"]

# Uncomment to see class names
# print("Base Class Names:", [(i, CLASS_NAMES[i]) for i in base_classes])
# print("Novel Class Names:", [(i, CLASS_NAMES[i]) for i in novel_classes])

In [5]:
# Load CLIP model and preprocessing
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device=device)
print(f"Device: {device}")
print(f"Model: ViT-B/16")

Device: cpu
Model: ViT-B/16


## Load Flowers102 and Split Base/Novel

We load the 3 splits (train, val, test) and divide into base/novel.

**Statistics:**
- Train Base: 10 images × 51 classes = 510 images
- Val Base: 10 images × 51 classes = 510 images
- Test Base: ~10 images × 51 classes (from test split)
- Test Novel: Remaining (~10 per class)

**Note:** Train and val have ~10 images per class (few-shot setting).


In [6]:
# get the three datasets
train_set, val_set, test_set = get_data(transform=preprocess)

# split classes into base and novel
base_classes, novel_classes = base_novel_categories(train_set)

# split the three datasets
train_base, _ = split_data(train_set, base_classes)
val_base, _ = split_data(val_set, base_classes)
test_base, test_novel = split_data(test_set, base_classes)

print(f"Train Base: {len(train_base)} samples")
print(f"Val Base: {len(val_base)} samples")
print(f"Test Base: {len(test_base)} samples")
print(f"Test Novel: {len(test_novel)} samples")

Train Base: 510 samples
Val Base: 510 samples
Test Base: 2473 samples
Test Novel: 3676 samples


## (Sezione rimossa) Zero-Shot CLIP Evaluation

Questa sezione è stata rimossa: il notebook esegue ora esclusivamente il workflow CoCoOp (prompt learning + MetaNetwork).

Se vuoi riattivare la valutazione zero-shot più tardi, esegui una cella separata che costruisca i prompt fissi e chiami `model.encode_text()`.


In [7]:
# Zero-shot evaluation removed — notebook runs CoCoOp only now.
# Se serve, riattivare la valutazione zero-shot separatamente.
print("Zero-shot evaluation removed. Proceeding with CoCoOp-only workflow.")


Zero-shot evaluation removed. Proceeding with CoCoOp-only workflow.


## Harmonic Mean (HM)

Standard metric for few-shot adaptation papers.

Formula: HM = 2 / (1/base_acc + 1/novel_acc)

**Why HM instead of arithmetic mean?**
- HM heavily penalizes outliers
- If base=90% and novel=50%: arithmetic mean=70%, HM=64.3%
- Forces the model to balance both accuracies

**Obiettivo:** massimizzare l'HM tra `base_acc_cocoop` e `novel_acc_cocoop`.


In [8]:
def harmonic_mean(base_accuracy, novel_accuracy):
    numerator = 2
    denominator = 1 / base_accuracy + 1 / novel_accuracy
    return numerator / denominator


## MetaNetwork: Conditional Token Generator

**Problem:** Fixed prompts don't adapt to each image.

**Solution:** A small neural network that transforms image features into a conditional token.

**Parameters:** ~256K (negligible vs. fine-tuning)

**Effect:** Each image gets a different prompt → instance-level adaptation


In [9]:
"""
MetaNetwork è una piccola rete neurale (MLP con 2 layer)
che trasforma le image_features (512-dim) in un token
condizionale (512-dim) usato in CoCoOp.

Questo token varia per ogni immagine, permettendo prompt
personalizzati per ogni input.
"""

class MetaNetwork(nn.Module):
    def __init__(self, ctx_dim=512, hidden_dim=256):
        """
        Args:
            ctx_dim: dimensione degli embeddings (512 per ViT-B/16)
            hidden_dim: dimensione dello strato nascosto
        """
        super().__init__()
        self.linear1 = nn.Linear(ctx_dim, hidden_dim)
        self.relu = nn.ReLU(inplace=True)
        self.linear2 = nn.Linear(hidden_dim, ctx_dim)

    def forward(self, image_features):
        """
        Args:
            image_features: tensor (B, ctx_dim) dalle immagini encodate
        
        Returns:
            conditional_token: tensor (B, ctx_dim)
        """
        # Assicura il tipo corretto (importante per mixed precision)
        image_features = image_features.to(self.linear1.weight.dtype)
        
        out = self.linear1(image_features)
        out = self.relu(out)
        out = self.linear2(out)
        return out


## CoCoOpPromptLearner: Dynamic Prompts


**Components:**
1. **V1...VM:** 16 context vectors (learned via SGD)
   - Shape: (16, 512) tensors
   - Initialized randomly from N(0, 0.02²)
   - Optimized during training

2. **π(x):** Conditional token (generated per image)
   - Shape: (B, 512) from MetaNetwork output
   - Different for each image

3. **[CLASS]:** Class name embedding
   - Shape: (seq_len, 512) from CLIP's token embedding
   - Same for all images of the same class

**Forward Pass:**
- Input: image_features (B, 512)
- Output: prompts (B, num_classes, seq_len_total, 512)


In [10]:
class CoCoOpPromptLearner(nn.Module):
    def __init__(self, clip_model, classnames, n_ctx=4):  # ← 4 context tokens
        super().__init__()
        self.n_ctx = n_ctx
        self.classnames = classnames
        dtype = clip_model.dtype
        ctx_dim = int(clip_model.ln_final.weight.shape[0])
        self.clip_context_length = clip_model.context_length
        print(f"[CoCoOp] ctx_dim={ctx_dim}, max_len={self.clip_context_length}")
        
        device = next(clip_model.parameters()).device
        # 4 context vectors: ensure correct device and dtype
        ctx_vectors = torch.empty(n_ctx, ctx_dim, dtype=dtype, device=device)
        nn.init.normal_(ctx_vectors, std=0.02)
        self.ctx = nn.Parameter(ctx_vectors)
        
        # MetaNetwork created on same device/dtype as CLIP internals to avoid casts
        self.meta_net = MetaNetwork(ctx_dim).to(device=device, dtype=dtype)
        
        classnames_tokens = clip.tokenize(classnames).to(device)
        with torch.no_grad():
            # Ensure class_token_embeddings stored with the CLIP dtype
            self.register_buffer("class_token_embeddings", 
                               clip_model.token_embedding(classnames_tokens).to(dtype=dtype))
            # Save token ids for each class sequence so we can find the EOT position later
            self.register_buffer("class_token_ids", classnames_tokens)
        
    def forward(self, image_features):
        batch_size = image_features.shape[0]
        num_classes, class_len, ctx_dim = self.class_token_embeddings.shape
        
        # Conditional token per immagine (meta_net already on correct device/dtype)
        cond = self.meta_net(image_features).unsqueeze(1).unsqueeze(2)  # B,1,1,D
        cond = cond.repeat(1, num_classes, 1, 1)                         # B,N,1,D
        
        # Context (self.ctx already on correct device/dtype)
        ctx = self.ctx.unsqueeze(0).unsqueeze(0).repeat(batch_size, num_classes, 1, 1)  # B,N,4,D
        
        # Class
        cls_emb = self.class_token_embeddings.unsqueeze(0).repeat(batch_size, 1, 1, 1)  # B,N,L,D
        
        # Concat: [ctx] + [π(x)] + [class] 
        prompts = torch.cat([ctx, cond, cls_emb], dim=2)[:,:,:self.clip_context_length,:]
        return prompts


## CoCoOpTrainer: Training and Evaluation

Class that manages:

**1. Initialization:**
- Create PromptLearner
- Freeze CLIP (`requires_grad=False`)
- Configure SGD optimizer for prompt learner only

**2. train_epoch():**
- Forward: Image encoder + PromptLearner + Text encoder
- **Critical step:** Encode soft prompts through text transformer
  - Add positional embeddings
  - Pass through CLIP's transformer
  - Extract first token
  - Apply final layer norm + projection
- Compute loss: Cross-entropy on base classes
- Backward: Backprop only in PromptLearner
- Return: Average loss of the epoch

**3. eval():**
- Same forward procedure as training
- Without backward pass
- Compute accuracy on any dataset (base or novel)

**Important note:** We don't use `model.encode_text()` on soft prompts
because that method expects integer tokens, not embeddings.
We manually forward through the text transformer.

In [ ]:
"""
CoCoOpTrainer fornisce:
1. train_epoch(): esegue un epoca di training su base classes
2. eval(): valuta su base o novel classes

Importante: CLIP rimane congelato, alleniamo solo i prompt learner!
"""

class CoCoOpTrainer:
    def __init__(self, clip_model, base_classnames, base_classes, 
                 novel_classes, device, lr=0.002, chunk_size=128):
        """
        Args:
            clip_model: modello CLIP caricato
            base_classnames: lista di nomi classi base
            base_classes: lista di indici base classes
            novel_classes: lista di indici novel classes
            device: "cuda" o "cpu"
            lr: learning rate
            chunk_size: numero di sequenze da processare per chunk al transformer
        """
        self.clip_model = clip_model
        self.base_classnames = base_classnames
        self.base_classes = base_classes
        self.novel_classes = novel_classes
        self.device = device
        self.chunk_size = chunk_size
        
        # Precompute mapping from original class id -> contiguous index for base classes
        self.contig_cat2idx = {cat: idx for idx, cat in enumerate(self.base_classes)}
        
        # Freeze CLIP parameters
        for p in clip_model.parameters():
            p.requires_grad = False
        
        # Crea il prompt learner (messo sullo stesso device e dtype del modello CLIP)
        self.prompt_learner = CoCoOpPromptLearner(
            clip_model, 
            base_classnames
        ).to(device=device, dtype=clip_model.dtype)
        
        # Optimizer - allena solo il prompt learner
        self.optimizer = torch.optim.SGD(
            self.prompt_learner.parameters(),
            lr=lr,
            momentum=0.9,
            weight_decay=5e-4
        )
        
        # Mixed precision scaler (solo se CUDA disponibile)
        self.use_amp = ("cuda" in str(self.device)) and torch.cuda.is_available()
        self.scaler = GradScaler() if self.use_amp else None
        
    def train_epoch(self, train_dataset, batch_size=32):
        """
        Esegue una epoca di training con chunking e mixed-precision.
        """
        self.prompt_learner.train()
        self.clip_model.eval()
        
        dataloader = torch.utils.data.DataLoader(
            train_dataset, 
            batch_size=batch_size, 
            shuffle=True, 
            num_workers=0
        )
        
        total_loss = 0
        n_batches = 0
        
        for batch_idx, (images, labels) in enumerate(tqdm(dataloader, desc="CoCoOp training")):
            images = images.to(self.device)
            labels = labels.to(self.device)
            
            # Encode immagini (frozen)
            with torch.no_grad():
                img_feat = self.clip_model.encode_image(images)
            
            # Normalizza image features e porta al dtype del prompt learner
            img_feat = img_feat.to(self.prompt_learner.meta_net.linear1.weight.dtype)
            img_feat /= img_feat.norm(dim=-1, keepdim=True)
            
            # Genera prompts condizionali per ogni immagine
            prompts = self.prompt_learner(img_feat)  # (B, num_classes, seq_len, ctx_dim)
            B, N, L, D = prompts.shape
            prompts_flat = prompts.view(B * N, L, D)
            
            # Prepare token ids and EOT base positions
            class_indices = torch.arange(N, device=self.prompt_learner.class_token_ids.device).unsqueeze(0).repeat(B, 1).view(-1)
            token_ids = self.prompt_learner.class_token_ids[class_indices]  # (B*N, L)
            eot_base = token_ids.argmax(dim=-1)  # (B*N,)
            offset = self.prompt_learner.n_ctx + 1
            eot_positions = eot_base + offset  # (B*N,)
            
            # Chunked transformer forward to extract only EOT outputs
            pos_emb = self.clip_model.positional_embedding.to(device=prompts_flat.device, dtype=prompts_flat.dtype)
            outs = []
            for i in range(0, prompts_flat.size(0), self.chunk_size):
                chunk = prompts_flat[i:i+self.chunk_size]              # (C, L, D)
                x_chunk = chunk + pos_emb[:chunk.size(1)]
                x_chunk = x_chunk.permute(1, 0, 2)                    # (L, C, D)
                x_chunk = self.clip_model.transformer(x_chunk)
                x_chunk = x_chunk.permute(1, 0, 2)                    # (C, L, D)
                # slice corresponding eot positions and clamp
                eot_chunk = eot_positions[i:i+chunk.size(0)]
                eot_chunk = torch.clamp(eot_chunk, max=x_chunk.shape[1]-1)
                out_chunk = x_chunk[torch.arange(x_chunk.shape[0], device=x_chunk.device), eot_chunk.to(x_chunk.device)]  # (C, D)
                outs.append(out_chunk)
            x = torch.cat(outs, dim=0)  # (B*N, D)
            
            # Layer norm finale e projection
            x = self.clip_model.ln_final(x)
            text_proj = self.clip_model.text_projection.to(device=x.device, dtype=x.dtype)
            
            # Autocast for mixed precision if available
            if self.use_amp:
                with autocast():
                    text_feat = (text_proj @ x.T).T  # (B*N, 512)
                    text_feat = text_feat.view(B, N, -1)
                    text_feat /= text_feat.norm(dim=-1, keepdim=True)
                    logit_scale = self.clip_model.logit_scale.exp()
                    logits = logit_scale * (img_feat.unsqueeze(1) * text_feat).sum(-1)
                    labels_mapped = torch.tensor([self.contig_cat2idx[l.item()] for l in labels], dtype=torch.long, device=self.device)
                    loss = F.cross_entropy(logits, labels_mapped)
                # backward with scaler
                self.optimizer.zero_grad()
                self.scaler.scale(loss).backward()
                self.scaler.step(self.optimizer)
                self.scaler.update()
            else:
                text_feat = (text_proj @ x.T).T  # (B*N, 512)
                text_feat = text_feat.view(B, N, -1)
                text_feat /= text_feat.norm(dim=-1, keepdim=True)
                logit_scale = self.clip_model.logit_scale.exp()
                logits = logit_scale * (img_feat.unsqueeze(1) * text_feat).sum(-1)
                labels_mapped = torch.tensor([self.contig_cat2idx[l.item()] for l in labels], dtype=torch.long, device=self.device)
                loss = F.cross_entropy(logits, labels_mapped)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

            total_loss += loss.item()
            n_batches += 1

            # free intermediate memory if CUDA
            if self.use_amp:
                torch.cuda.empty_cache()
        
        avg_loss = total_loss / max(1, n_batches)
        return avg_loss
    
    @torch.no_grad()
    def eval(self, dataset, categories, batch_size=64):
        """
        Valuta il modello su un dataset usando chunking e autocast per risparmiare memoria.
        """
        self.prompt_learner.eval()
        self.clip_model.eval()
        
        contig_cat2idx = {cat: idx for idx, cat in enumerate(categories)}
        
        dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=0
        )
        
        correct = 0
        total = 0
        
        for images, labels in tqdm(dataloader, desc="CoCoOp eval"):
            images = images.to(self.device)
            labels = labels.to(self.device)
            
            img_feat = self.clip_model.encode_image(images)
            img_feat = img_feat.to(self.prompt_learner.meta_net.linear1.weight.dtype)
            img_feat /= img_feat.norm(dim=-1, keepdim=True)
            
            prompts = self.prompt_learner(img_feat)
            B, N, L, D = prompts.shape
            prompts_flat = prompts.view(B * N, L, D)

            class_indices = torch.arange(N, device=self.prompt_learner.class_token_ids.device).unsqueeze(0).repeat(B, 1).view(-1)
            token_ids = self.prompt_learner.class_token_ids[class_indices]
            eot_base = token_ids.argmax(dim=-1)
            offset = self.prompt_learner.n_ctx + 1
            eot_positions = eot_base + offset

            pos_emb = self.clip_model.positional_embedding.to(device=prompts_flat.device, dtype=prompts_flat.dtype)
            outs = []
            for i in range(0, prompts_flat.size(0), self.chunk_size):
                chunk = prompts_flat[i:i+self.chunk_size]
                x_chunk = chunk + pos_emb[:chunk.size(1)]
                x_chunk = x_chunk.permute(1, 0, 2)
                # autocast in eval if available
                if self.use_amp:
                    with autocast():
                        x_chunk = self.clip_model.transformer(x_chunk)
                else:
                    x_chunk = self.clip_model.transformer(x_chunk)
                x_chunk = x_chunk.permute(1, 0, 2)
                eot_chunk = eot_positions[i:i+chunk.size(0)]
                eot_chunk = torch.clamp(eot_chunk, max=x_chunk.shape[1]-1)
                out_chunk = x_chunk[torch.arange(x_chunk.shape[0], device=x_chunk.device), eot_chunk.to(x_chunk.device)]
                outs.append(out_chunk)
            x = torch.cat(outs, dim=0)

            x = self.clip_model.ln_final(x)
            text_proj = self.clip_model.text_projection.to(device=x.device, dtype=x.dtype)
            if self.use_amp:
                with autocast():
                    text_feat = (text_proj @ x.T).T
            else:
                text_feat = (text_proj @ x.T).T
            text_feat = text_feat.view(B, N, -1)
            text_feat /= text_feat.norm(dim=-1, keepdim=True)

            logit_scale = self.clip_model.logit_scale.exp()
            logits = logit_scale * (img_feat.unsqueeze(1) * text_feat).sum(-1)

            pred = logits.argmax(dim=1)
            labels_mapped = torch.tensor([contig_cat2idx[l.item()] for l in labels], dtype=torch.long, device=self.device)
            correct += (pred == labels_mapped).sum().item()
            total += labels.size(0)

        return correct / total


: 

## Training CoCoOp

We will train the PromptLearner for **10 epochs** on **base classes only**.

**Hyperparameters:**
- Learning rate: 0.002 (SGD)
- Momentum: 0.9
- Weight decay: 5e-4
- Batch size: 32
- Epochs: 10

**What happens:**
- Context vectors V1...VM adapt to the Flowers102 dataset
- MetaNetwork learns to generate useful conditional tokens
- CLIP remains frozen (unchanged)

**Expected output:**
- Initial loss: ~3.0
- Final loss: ~1.3-1.5
- Training time: ~5-10 minutes on GPU

In [12]:

# Preparing base class names for CoCoOp
base_classnames = [CLASS_NAMES[i] for i in base_classes]
print(f"Base classnames ({len(base_classnames)}): {base_classnames[:5]}...\n")

# Creating trainer for CoCoOp
trainer = CoCoOpTrainer(
    clip_model=model,
    base_classnames=base_classnames,
    base_classes=base_classes,
    novel_classes=novel_classes,
    device=device,
    lr=0.002
)

num_epochs = 10
for epoch in range(num_epochs):
    avg_loss = trainer.train_epoch(train_base, batch_size=32)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")



Base classnames (51): ['pink primrose', 'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea', 'english marigold']...

[CoCoOp] ctx_dim=512, max_len=77


CoCoOp training:   0%|          | 0/16 [00:00<?, ?it/s]

: 

: 

## Final Evaluation (CoCoOp only)

Valuteremo il modello addestrato su:
1. Test Base — classi viste durante l'allenamento
2. Test Novel — classi non viste

Calcoliamo l'Harmonic Mean tra le due per valutare il trade-off.


In [ ]:
print("\n" + "="*60)
print("EVALUATION AND COMPARISON (CoCoOp Only)")
print("="*60)

# Valutazione CoCoOp su base e novel
base_acc_cocoop = trainer.eval(test_base, base_classes, batch_size=64)
novel_acc_cocoop = trainer.eval(test_novel, novel_classes, batch_size=64)
hm_cocoop = harmonic_mean(base_acc_cocoop, novel_acc_cocoop)

# Stampa risultati CoCoOp
print("\n" + "="*60)
print("CoCoOp RESULTS")
print("="*60)

print(f"   Base Accuracy:  {base_acc_cocoop*100:6.2f}%")
print(f"   Novel Accuracy: {novel_acc_cocoop*100:6.2f}%")
print(f"   Harmonic Mean:  {hm_cocoop*100:6.2f}%")

print("\n" + "="*60)
